In [2]:
import sqlite3
import pandas as pd

In [3]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [4]:
conn = sqlite3.connect('parch-and-posey.db')

In [5]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [6]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,92,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,93,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,94,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,221,"CREATE TABLE accounts (\tid integer,\tname bpc..."


# NTILE(*# of buckets*)

You can use window functions to identify what percentile (or quartile, or any other subdivision) a given row falls into. The syntax is NTILE(*# of buckets*). In this case, ORDER BY determines which column to use to determine the quartiles (or whatever number of ‘tiles you specify).

Expert Tip
In cases with relatively few rows in a window, the NTILE function doesn’t calculate exactly as you might expect. For example, If you only had two records and you were measuring percentiles, you’d expect one record to define the 1st percentile, and the other record to define the 100th percentile. Using the NTILE function, what you’d actually see is one record in the 1st percentile, and one in the 2nd percentile.

In other words, when you use a NTILE function but the number of rows in the partition is less than the NTILE(number of groups), then NTILE will divide the rows into as many groups as there are members (rows) in the set but then stop short of the requested number of groups. If you’re working with very small windows, keep this in mind and consider using quartiles or similarly small bands.

You can check out a complete list of window functions in Postgres (the syntax Mode uses) in the Postgres documentation.
https://www.postgresql.org/docs/8.4/functions-window.html

In [13]:
pd.read_sql_query(sql='''
SELECT id, standard_qty,
NTILE(4) OVER main_window quartile_standard_qty,
NTILE(5) OVER main_window quintile_standard_qty,
NTILE(100) OVER main_window percentile_standard_qty
FROM orders
WINDOW main_window AS (ORDER BY standard_qty);
''', con=conn)

,id,standard_qty,quartile_standard_qty,quintile_standard_qty,percentile_standard_qty
0,119,0,1,1,1
1,124,0,1,1,1
2,196,0,1,1,1
3,254,0,1,1,1
4,328,0,1,1,1
...,...,...,...,...,...
6907,1112,6043,4,5,100
6908,5167,7083,4,5,100
6909,5479,7365,4,5,100
6910,4562,15649,4,5,100
